[View in Colaboratory](https://colab.research.google.com/github/jimmyjamesarnold/10k_statistical_inferences/blob/master/10k_game_stats.ipynb)

10000 Dice Game in Python - when should you 'lock in' your score?

In [0]:
# 10000 in Python (for stats)
# single game, threshold = 200
# dice df generator 

# dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
from itertools import chain, combinations

In [0]:
def rollDice(n) :
  """ takes input n # dice to roll, returns pd.df"""
  rollList = pd.DataFrame([[n+1,np.random.randint(1, 6)] for n in range(n)])
  rollList.columns = ['die','face']
  return rollList

def searchOnes(df):
  """pass in df from diceRoll, evaluates, saves scores and dice indices"""
  matched = len(df[df.face == 1].face)
  dice = df[df.face == 1].index.values
  if matched == 3: 
      options.append([1000, dice])
  elif matched > 3: 
      options.append([1000 * (2 ** (matched - 3)), dice])

def nOfaKind(df): 
  """pass in df from diceRoll, evaluates, saves scores and dice indices"""
  for i in range(2,7): 
      matched = len(df[df.face == i].face)
      dice = df[df.face == i].index.values         
      if matched == 3: 
          options.append([i*100, dice])
      elif matched > 3: 
          options.append([(i*100) * (2 ** (matched - 3)), dice])

def straight(df):# looks for straight
  """ looks for straight """
  if df.face.nunique() == 6:
      options.append([1000, df.index.values])

# Looks for 3 Pairs
def threePairs(df):
  """ Counts matched pairs in set, looks for 3 pairs of 2"""
  if list(Counter([(i,j) for i in df.die for j in df.face if i == j]).values()).count(2) == 3: # test for 3 sets of 2 in the list of counts of frequency of pairs
      options.append([1000, df.index.values])            

def comboSearch(df):
  """ search for scoring combos in set """
  searchOnes(df)
  nOfaKind(df)
  straight(df)
  threePairs(df)
        

In [16]:
scoreKeeper = [] # Saves top scores for stats
#gameRates = [] # Saves scoring history - I want to plot later.
gameLengths = [] # Store time to 10000, ### need to make a conditional turn counter
for i in range(10): 
  gameScore = [0] # this games' score
  turnsTo10k = 0 # turn counter
  for rounds in range(30): #run the game for 30 cycles, see what scores are generated
    if gameScore[-1] <= 10000:
      turnsTo10k += 1
    score = [0] #this round's score
    n = 6 # initialize n dice (6)
    while n > 0:
        options = [] # initialize options for storing combos, inside while so it's always fresh
        df = rollDice(n) #df of dice and faces
        comboSearch(df) 
        dfOpt = pd.DataFrame(options, columns = ['points', 'dice'])
        if sum(dfOpt.points) > 0 : # check for combos
            dfOpt = dfOpt[dfOpt.points == max(dfOpt.points)].reset_index() # subset for max points
            score = sum(score + dfOpt.points) # record score from sets
            df = df.drop(dfOpt.dice[0]) # drop scored dice
        else: score = 0 
        del options[:] # delete options
        dfOpt = pd.DataFrame(columns=dfOpt.columns) # clears dfOpt
        if len(df.index.values) > 0: # checks remaining dice for 1s and 5s
            extra = sum([100*len(df[df.face == 1].face)] + [50*len(df[df.face == 5].face)]) # count up 100s and 50s
            score = score + extra # add extra points
            df = df.drop(df[df.face == 1].index.values)
            df = df.drop(df[df.face == 5].index.values)     # drops remaining 1s and 5s
        if score == 0: # You rolled nothing
            gameScore.append(gameScore[-1])
            n = 0 # ends turn
        elif score >= 200 and gameScore[-1] >= 1000: # Hit "threshold", lock in 
            gameScore.append(gameScore[-1] + score)
            n = 0 # ends turn
        elif score >= 1000: #You're on the board
            gameScore.append(gameScore[-1] + score)
            n = 0 # ends turn
        else: # threshold not hit, re-roll dice
            if len(df.index.values) > 0:
              n = len(df.index.values) # re-roll remaining dice.
            else: n = 6 # All dice were used, but threshold not hit, roll 6 new dice
  scoreKeeper.append(gameScore[-1]) # save highest score
  #gameRates.append(gameScore) # save scoring pattern
  gameLengths.append(turnsTo10k) # save turns to 10k
  print(gameScore)
  
  

[0, 0, 1050, 1250, 1450, 1650, 2200, 2500, 2700, 3850, 4150, 4350, 4800, 5000, 5400, 5700, 5950, 6850, 7050, 7300, 7300, 7500, 8000, 8500, 8700, 9850, 10100, 10750, 11250, 11500, 12500]
[0, 0, 0, 1000, 1500, 1950, 3050, 3550, 3800, 4250, 4750, 4950, 5950, 6150, 6150, 6650, 7650, 8650, 9650, 9900, 10400, 11050, 12050, 12650, 12650, 13200, 13400, 13700, 14150, 14400, 15400]
[0, 0, 1100, 1300, 1550, 2650, 2850, 4900, 5150, 5150, 5350, 5550, 5800, 6100, 6950, 7150, 7350, 7950, 8300, 9350, 9550, 9800, 10050, 10750, 11800, 12100, 12300, 16300, 17350, 18000, 18450]
[0, 0, 0, 0, 1000, 1200, 1400, 1400, 1600, 2200, 2900, 2900, 3950, 4150, 4350, 4850, 5450, 5750, 6050, 6050, 6650, 6850, 7150, 7750, 8050, 9150, 9600, 10000, 10350, 11350, 11600]
[0, 1000, 1400, 2000, 2300, 2550, 2900, 3300, 3800, 4400, 4850, 5300, 6000, 7000, 7500, 7750, 8400, 9400, 10200, 10400, 11500, 12000, 12500, 12750, 13150, 14200, 14600, 15100, 16300, 16600, 17300]
[0, 0, 0, 0, 0, 0, 0, 1000, 1200, 1400, 1800, 2050, 2650, 2

In [24]:
print(str('final scores:'),scoreKeeper)
print(str('turns to 10000:'),gameLengths)

final scores: [12500, 15400, 18450, 11600, 17300, 10500, 14050, 15900, 19150, 14900]
turns to 10000: [26, 20, 22, 28, 18, 29, 23, 17, 18, 21]


In [31]:
threshold = list(range(100,1001,100))
allGameLengths = []
allHighScores = []
for pts in threshold:
  print(str('threshold: '),pts) # option to see if code is going
  scoreKeeper = [] # Saves top scores for stats
  #gameRates = [] # Saves scoring history - I want to plot later.
  gameLengths = [] # Store time to 10000, ### need to make a conditional turn counter
  for i in range(10): 
    gameScore = [0] # this games' score
    turnsTo10k = 0 # turn counter
    for rounds in range(30): #run the game for 30 cycles, see what scores are generated
      if gameScore[-1] <= 10000:
        turnsTo10k += 1
      score = [0] #this round's score
      n = 6 # initialize n dice (6)
      while n > 0:
          options = [] # initialize options for storing combos, inside while so it's always fresh
          df = rollDice(n) #df of dice and faces
          comboSearch(df) 
          dfOpt = pd.DataFrame(options, columns = ['points', 'dice'])
          if sum(dfOpt.points) > 0 : # check for combos
              dfOpt = dfOpt[dfOpt.points == max(dfOpt.points)].reset_index() # subset for max points
              score = sum(score + dfOpt.points) # record score from sets
              df = df.drop(dfOpt.dice[0]) # drop scored dice
          else: score = 0 
          del options[:] # delete options
          dfOpt = pd.DataFrame(columns=dfOpt.columns) # clears dfOpt
          if len(df.index.values) > 0: # checks remaining dice for 1s and 5s
              extra = sum([100*len(df[df.face == 1].face)] + [50*len(df[df.face == 5].face)]) # count up 100s and 50s
              score = score + extra # add extra points
              df = df.drop(df[df.face == 1].index.values)
              df = df.drop(df[df.face == 5].index.values)     # drops remaining 1s and 5s
          if score == 0: # You rolled nothing
              gameScore.append(gameScore[-1])
              n = 0 # ends turn
          elif score >= pts and gameScore[-1] >= 1000: # Hit "threshold", lock in 
              gameScore.append(gameScore[-1] + score)
              n = 0 # ends turn
          elif score >= 1000: #You're on the board
              gameScore.append(gameScore[-1] + score)
              n = 0 # ends turn
          else: # threshold not hit, re-roll dice
              if len(df.index.values) > 0:
                n = len(df.index.values) # re-roll remaining dice.
              else: n = 6 # All dice were used, but threshold not hit, roll 6 new dice
    print(str('round: '),i,str(' final score: '),gameScore[-1]) # option to see if code is going
    scoreKeeper.append(gameScore[-1]) # save highest score
    #gameRates.append(gameScore) # save scoring pattern
    gameLengths.append(turnsTo10k) # save turns to 10k
  allHighScores.append(scoreKeeper)
  allGameLengths.append(gameLengths)


threshold:  100
round:  0  final score:  9700
round:  1  final score:  16450
round:  2  final score:  12300
round:  3  final score:  14000
round:  4  final score:  11400
round:  5  final score:  12850
round:  6  final score:  11850
round:  7  final score:  8150
round:  8  final score:  11800
round:  9  final score:  15200
threshold:  200
round:  0  final score:  12800
round:  1  final score:  10800
round:  2  final score:  15400
round:  3  final score:  16700
round:  4  final score:  13200
round:  5  final score:  10800
round:  6  final score:  18100
round:  7  final score:  18400
round:  8  final score:  17150
round:  9  final score:  15400
threshold:  300
round:  0  final score:  15450
round:  1  final score:  12100
round:  2  final score:  16100
round:  3  final score:  16350
round:  4  final score:  10950
round:  5  final score:  17400
round:  6  final score:  17450
round:  7  final score:  15750
round:  8  final score:  12600
round:  9  final score:  15150
threshold:  400
round:  

round:  7  final score:  10950
round:  8  final score:  11050
round:  9  final score:  17250
threshold:  900
round:  0  final score:  8300
round:  1  final score:  14750
round:  2  final score:  13050
round:  3  final score:  8450
round:  4  final score:  11100
round:  5  final score:  11900
round:  6  final score:  9050
round:  7  final score:  12900
round:  8  final score:  5400
round:  9  final score:  11350
threshold:  1000
round:  0  final score:  10250
round:  1  final score:  6400
round:  2  final score:  12250
round:  3  final score:  17550
round:  4  final score:  12750
round:  5  final score:  5150
round:  6  final score:  8500
round:  7  final score:  9200
round:  8  final score:  10650
round:  9  final score:  7200


Just from a quick look at the final scores, it seems pretty clear that going above 500 seem to have a lower probability of reaching 10000. 
This can visualized via histogram of the 'allHighScores' - which tracked the highest score reached after 30 rounds, for each threshold.

In [29]:
print(allHighScores)
print(allGameLengths)

[[13000, 17700, 12150, 15900, 10800, 19500, 16500, 15650, 14100, 13750], [10500, 12600, 16350, 14900, 14850, 14550, 13700, 12600, 12850, 16900], [14850, 12000, 14750, 14650, 11650, 10500, 10400, 14450, 14650, 14500], [20600, 10050, 9900, 17150, 12600, 14100, 8300, 8800, 15150, 10900], [13750, 15000, 15950, 11050, 8600, 11050, 16450, 19500, 14400, 8850], [12100, 12100, 14800, 14500, 15200, 11200, 16900, 14200, 8950, 13350], [4650, 14350, 14250, 10900, 13050, 10550, 13450, 10400, 13700, 13900], [13150, 10450, 8400, 6900, 10700, 9600, 11400, 12700, 12650, 8950], [7750, 14750, 7500, 13050, 14900, 13600, 8800, 16150, 21950, 6850], [5200, 4400, 4150, 10000, 12650, 9600, 9000, 10100, 6850, 18900]]
[[25, 20, 26, 16, 30, 16, 15, 19, 22, 17], [29, 27, 18, 23, 15, 18, 22, 25, 25, 15], [23, 23, 22, 22, 22, 26, 30, 16, 22, 26], [9, 29, 30, 17, 27, 26, 30, 30, 19, 30], [19, 21, 16, 27, 30, 27, 26, 21, 24, 30], [29, 28, 16, 24, 25, 29, 19, 20, 30, 22], [30, 24, 22, 30, 25, 30, 28, 29, 19, 25], [23, 2

This can also be assessed by plotting the histograms of the 'time to 10k' counter, which reports at which turn 10k was reached for each threshold. 

In [0]:
plt.hist(scoreKeeper, bins = 10)
plt.show()